In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# TODO:3次元プロットで2変数の線形飽和モデルをプロットする

plotBase = list(range(20))
plotX = returnListForBranchModel(inputList=plotBase, branchIndex=13, a=2, b=3)
plotY = returnListForBranchModel(inputList=plotBase, branchIndex=16, a=4, b=5)
plotX = np.array(plotX)
plotY = np.array(plotY)
plotZ = plotX+plotY

# TODO:関数コール回数が同じだが、2変数が異なる変数かつそういったものが多数ある場合が分岐線

print(f"plotBase={plotBase}\nplotX={plotX}\nplotY={plotY}")

fig = plt.figure()
ax = Axes3D(fig)

ax.set_xlabel("X-axis")
ax.set_ylabel("Y-axis")
ax.set_zlabel("Z-axis")

plotZero = [0]*len(plotBase)
ax.scatter(plotBase, plotZero, plotX)
ax.scatter(plotZero, plotBase, plotY)

dataset01 = pd.DataFrame({"X": plotBase, "Y": plotZero, "Z": plotX})
dataset02 = pd.DataFrame({"X": plotZero, "Y": plotBase, "Z": plotY})

# 説明変数が2つの分岐モデルを実装するにあたってのテストなど

1. ~\[X, Y, Z\] == \[0, Y, Z\], \[X, 0, Z\] を満たすデータ群を抽出~
2. 抽出した2つのデータ群から分岐点を抽出
3. 抽出した分岐点を繋ぐ直線を定義
4. データ群を定義した直線で2つに分割
5. 分割された2平面ごとにモデルを構築

In [ ]:
print(dataset01.tail())

In [ ]:
print(dataset02.tail())

In [12]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

test_returnBranchIndexOfList()

inputListX=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], inputListY=[4, 7, 10, 13, 16, 19, 22, 25, 28, 31, 34, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37]
[2.99, 2.99, 2.99, 2.99, 2.99, 2.99, 2.99, 2.99, 2.99, 2.99, 1.49, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


AssertionError: 